In [21]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, './../src')

import data_loader
import data_references
from energy_channels import EnergyChannel
import enhancement_finder
import datetime
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import plot_tools
import numpy as np
import pandas as pd
import psd_tool
from field_models import model
import data_cache


import importlib
importlib.reload(plot_tools)
importlib.reload(data_loader)
importlib.reload(enhancement_finder)
importlib.reload(data_references)
importlib.reload(psd_tool)
importlib.reload(data_cache)

%matplotlib qt

In [2]:
start = datetime.datetime(year = 2013, month = 1, day = 1)
end = datetime.datetime(year = 2013, month = 12, day = 31, hour=23, minute=59, second=59)

In [3]:

dataRefsA = data_loader.load_compressed_rept_data(satellite="A", start=start, end=end)
dataRefsB = data_loader.load_compressed_rept_data(satellite="B", start=start, end=end)

fesa_A, L_A, mlt_A, epoch_A, energies = dataRefsA.get_all_data()
fesa_B, L_B, mlt_B, epoch_B, energies = dataRefsB.get_all_data()

omni = data_loader.load_omni_data(start=start, end=end)


Loading REPT data between: 2013-01-01 00:00:00 and 2013-12-31 23:59:59.
Loading : REPT_201301_A.npz
Loading : REPT_201302_A.npz
Loading : REPT_201303_A.npz
Loading : REPT_201304_A.npz
Loading : REPT_201305_A.npz
Loading : REPT_201306_A.npz
Loading : REPT_201307_A.npz
Loading : REPT_201308_A.npz
Loading : REPT_201309_A.npz
Loading : REPT_201310_A.npz
Loading : REPT_201311_A.npz
Loading : REPT_201312_A.npz
Loading REPT data between: 2013-01-01 00:00:00 and 2013-12-31 23:59:59.
Loading : REPT_201301_B.npz
Loading : REPT_201302_B.npz
Loading : REPT_201303_B.npz
Loading : REPT_201304_B.npz
Loading : REPT_201305_B.npz
Loading : REPT_201306_B.npz
Loading : REPT_201307_B.npz
Loading : REPT_201308_B.npz
Loading : REPT_201309_B.npz
Loading : REPT_201310_B.npz
Loading : REPT_201311_B.npz
Loading : REPT_201312_B.npz
Loading OMNI data between: 2013-01-01 00:00:00 and 2013-12-31 23:59:59
Loaded OMNI Data for : 2013-01-01 00:00:00
Loaded OMNI Data for : 2013-02-01 00:00:00
Loaded OMNI Data for : 2013

In [4]:
l_cut = 5.0
tol = 0.25

satisfies_L_cut_A = ((l_cut - tol) < L_A) & (L_A < (l_cut + tol))
satisfies_L_cut_B = ((l_cut - tol) < L_B) & (L_B < (l_cut + tol))

fesa_A = fesa_A[:, (energies < 6.5)]
fesa_B = fesa_B[:, (energies < 6.5)]

fesa_A_DF = pd.DataFrame(fesa_A[satisfies_L_cut_A, :], index=epoch_A[satisfies_L_cut_A])
fesa_B_DF = pd.DataFrame(fesa_B[satisfies_L_cut_B, :], index=epoch_B[satisfies_L_cut_B])

In [27]:
fesa = pd.concat([fesa_A_DF, fesa_B_DF], sort=False).sort_index()
fesa = fesa.resample("5h").mean()
#FESA = FESA.rolling(window=avg_t, center=True).mean()

In [6]:
metop1 = data_loader.load_compressed_poes_data("metop1", start=start, end=end)
metop2 = data_loader.load_compressed_poes_data("metop2", start=start, end=end)
noaa15 = data_loader.load_compressed_poes_data("noaa15", start=start, end=end)
noaa16 = data_loader.load_compressed_poes_data("noaa16", start=start, end=end)
noaa18 = data_loader.load_compressed_poes_data("noaa18", start=start, end=end)
noaa19 = data_loader.load_compressed_poes_data("noaa19", start=start, end=end)

satellites = [metop1, metop2, noaa15, noaa16, noaa18, noaa19]


Loading : POES_201301_metop1_CLEAN.npz
Loading : POES_201302_metop1_CLEAN.npz
Loading : POES_201303_metop1_CLEAN.npz
Loading : POES_201304_metop1_CLEAN.npz
Loading : POES_201305_metop1_CLEAN.npz
Loading : POES_201306_metop1_CLEAN.npz
Loading : POES_201307_metop1_CLEAN.npz
Loading : POES_201308_metop1_CLEAN.npz
Loading : POES_201309_metop1_CLEAN.npz
Loading : POES_201310_metop1_CLEAN.npz
Loading : POES_201311_metop1_CLEAN.npz
Loading : POES_201312_metop1_CLEAN.npz
Loading : POES_201301_metop2_CLEAN.npz
Loading : POES_201302_metop2_CLEAN.npz
Loading : POES_201303_metop2_CLEAN.npz
Loading : POES_201304_metop2_CLEAN.npz
Loading : POES_201305_metop2_CLEAN.npz
Loading : POES_201306_metop2_CLEAN.npz
Loading : POES_201307_metop2_CLEAN.npz
Loading : POES_201308_metop2_CLEAN.npz
Loading : POES_201309_metop2_CLEAN.npz
Loading : POES_201310_metop2_CLEAN.npz
Loading : POES_201311_metop2_CLEAN.npz
Loading : POES_201312_metop2_CLEAN.npz
Loading : POES_201301_noaa15_CLEAN.npz
Loading : POES_201302_noa

In [7]:
poes_epoch = np.concatenate([sat.get_data(data_references.POES.EPOCH) for sat in satellites], axis=0)
poes_L = np.concatenate([sat.get_data(data_references.POES.L) for sat in satellites], axis = 0)
chorus = np.concatenate([sat.get_data(data_references.POES.NAIVE_CHORUS_AMPLITUDES) for sat in satellites], axis=0)

satisfies_L_cut_POES = ((l_cut - tol) < poes_L) & (poes_L < (l_cut + tol))
poes_epoch = poes_epoch[satisfies_L_cut_POES]
poes_L = poes_L[satisfies_L_cut_POES]
chorus = chorus[satisfies_L_cut_POES]

POES = pd.DataFrame({"L": poes_L, "Chorus": chorus}, index=poes_epoch).sort_index().resample("1h").mean()

In [8]:
field_model = model.TS04D
mu_vals = [500, 1000, 3000, 5000, 8000]
chosen_k = 0.100

In [9]:
psd_refs_A = data_loader.load_psd(satellite = "A", field_model = field_model, start = start, end = end)
psd_refs_B = data_loader.load_psd(satellite = "B", field_model = field_model, start = start, end = end)

Loading : PSD_201301_A_TS04D.npz
Loading : PSD_201302_A_TS04D.npz
Loading : PSD_201303_A_TS04D.npz
Loading : PSD_201304_A_TS04D.npz
Loading : PSD_201305_A_TS04D.npz
Loading : PSD_201306_A_TS04D.npz
Loading : PSD_201307_A_TS04D.npz
Loading : PSD_201308_A_TS04D.npz
Loading : PSD_201309_A_TS04D.npz
Loading : PSD_201310_A_TS04D.npz
Loading : PSD_201311_A_TS04D.npz
Loading : PSD_201312_A_TS04D.npz
Loading : PSD_201301_B_TS04D.npz
Loading : PSD_201302_B_TS04D.npz
Loading : PSD_201303_B_TS04D.npz
Loading : PSD_201304_B_TS04D.npz
Loading : PSD_201305_B_TS04D.npz
Loading : PSD_201306_B_TS04D.npz
Loading : PSD_201307_B_TS04D.npz
Loading : PSD_201308_B_TS04D.npz
Loading : PSD_201309_B_TS04D.npz
Loading : PSD_201310_B_TS04D.npz
Loading : PSD_201311_B_TS04D.npz
Loading : PSD_201312_B_TS04D.npz


In [10]:
for chosen_mu in mu_vals:
    
    print(f"Selecting PSD for A at mu = {chosen_mu}, k = {chosen_k}")
    
    selected_psd_refs_A = psd_tool.select_mu_and_k_from_psd(refs=psd_refs_A, chosen_mu=chosen_mu, chosen_k=chosen_k, debug_mode=False, verbose=False)
    
    print(f"Selecting PSD for B at mu = {chosen_mu}, k = {chosen_k}")

    selected_psd_refs_B = psd_tool.select_mu_and_k_from_psd(refs=psd_refs_B, chosen_mu=chosen_mu, chosen_k=chosen_k, debug_mode=False, verbose=False)

    data_cache.cache_psd_at_selected_mu_and_k(selected_psd_refs_A, satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    data_cache.cache_psd_at_selected_mu_and_k(selected_psd_refs_B, satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)


Selecting PSD for A at mu = 500, k = 0.1
Time taken for loop: 58.366305351257324
Selecting PSD for B at mu = 500, k = 0.1
Time taken for loop: 80.39964270591736
A_135702000000000_138855599900000_50000000_10000_TS04D.npz
B_135702000000000_138855599900000_50000000_10000_TS04D.npz
Selecting PSD for A at mu = 1000, k = 0.1
Time taken for loop: 52.86145734786987
Selecting PSD for B at mu = 1000, k = 0.1
Time taken for loop: 52.99232459068298
A_135702000000000_138855599900000_100000000_10000_TS04D.npz
B_135702000000000_138855599900000_100000000_10000_TS04D.npz
Selecting PSD for A at mu = 3000, k = 0.1
Time taken for loop: 52.89956831932068
Selecting PSD for B at mu = 3000, k = 0.1
Time taken for loop: 52.488468647003174
A_135702000000000_138855599900000_300000000_10000_TS04D.npz
B_135702000000000_138855599900000_300000000_10000_TS04D.npz
Selecting PSD for A at mu = 5000, k = 0.1
Time taken for loop: 52.64184260368347
Selecting PSD for B at mu = 5000, k = 0.1
Time taken for loop: 52.428474426

In [16]:

slow_moving = omni["AE"].rolling("3h", center=False).mean()
#f = (OMNI["AE"] - slow_moving)
f = omni["AE"].diff()
threshold = 50
shifted_f1 = (f-threshold)
shifted_f2 = (f+threshold)

zero_crossings1 = ((shifted_f1.shift() >= 0) & (shifted_f1 < 0)) | ((shifted_f1.shift() < 0) & (shifted_f1 >= 0))
zero_crossings2 = ((shifted_f2.shift() >= 0) & (shifted_f2 < 0)) | ((shifted_f2.shift() < 0) & (shifted_f2 >= 0))
zero_crossings = zero_crossings1 | zero_crossings2
num_crossings = zero_crossings.rolling("3h", center=False).sum().rolling("3h", center=True).mean()

fig, axs = plt.subplots(5, 1, sharex=True)

num_energies = fesa.shape[1]
energy_colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):
    
    axs[0].semilogy(fesa.index, fesa[E], label=f"{energies[E]} MeV", color=energy_colors(E), linewidth=2.5)
    
axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED")

num_mu = len(mu_vals)
mu_colors = plt.cm.get_cmap("viridis", num_mu)

for k, chosen_mu in enumerate(mu_vals):
    
    selected_psd_refs_A = data_cache.load_psd_cache(satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    selected_psd_refs_B = data_cache.load_psd_cache(satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    
    epoch_A = selected_psd_refs_A["EPOCH"]
    Lstar_A = selected_psd_refs_A["L_STAR"] 
    L_A = selected_psd_refs_A["L"]
    PSD_A = selected_psd_refs_A["PSD"]
    
    epoch_B = selected_psd_refs_B["EPOCH"]
    Lstar_B = selected_psd_refs_B["L_STAR"]
    L_B = selected_psd_refs_B["L"]
    PSD_B = selected_psd_refs_B["PSD"]
    
    #satisfies_L_cut_A = ((l_cut - tol) < Lstar_A) & (Lstar_A < (l_cut + tol))
    #satisfies_L_cut_B = ((l_cut - tol) < Lstar_B) & (Lstar_B < (l_cut + tol))

    satisfies_L_cut_A = ((l_cut - tol) < L_A) & (L_A < (l_cut + tol))
    satisfies_L_cut_B = ((l_cut - tol) < L_B) & (L_B < (l_cut + tol))
    
    epoch_A_cut = epoch_A[satisfies_L_cut_A]
    PSD_A_cut = PSD_A[satisfies_L_cut_A]
    
    epoch_B_cut = epoch_B[satisfies_L_cut_B]
    PSD_B_cut = PSD_B[satisfies_L_cut_B]
        
    PSD_A_DF = pd.Series(PSD_A_cut, index=epoch_A_cut)
    PSD_A_DF = PSD_A_DF[np.isfinite(PSD_A_DF)]

    PSD_B_DF = pd.Series(PSD_B_cut, index=epoch_B_cut)
    PSD_B_DF = PSD_B_DF[np.isfinite(PSD_B_DF)]

    PSD = pd.concat([PSD_A_DF, PSD_B_DF], sort=False).sort_index()
    PSD = PSD.resample("5h").mean()
    PSD = PSD[np.isfinite(PSD)]
    
    axs[1].semilogy(PSD.index, PSD, linewidth=2, color = mu_colors(k), label=f"{int(chosen_mu)} (MeV / G)")
    
    
axs[1].set_title(rf"RBSP-COMBINED, K = {chosen_k} ($R_E$$G^{{1/2}}$), {field_model.name}")
axs[1].set_ylabel("PSD (c/Mev/G)")
axs[1].legend()

axs[2].yaxis.grid(color='gray', linestyle='dashed')
axs[2].xaxis.grid(color='gray', linestyle='dashed')
axs[2].plot(num_crossings.index, num_crossings, color="black", label="Crossings")
axs[2].set_ylabel("# Crossing Threshold")
axs[2].set_ylim(bottom=0)
#axs[1].hlines(y=100, xmin=start, xmax=end, color="green", label="Enhancement Trigger")

axs[3].plot(POES.index, POES["Chorus"], color="black")
axs[3].set_ylabel("Naive Chorus (pT)")
axs[3].set_xlabel("Time")


axs[4].yaxis.grid(color='gray', linestyle='dashed')
axs[4].xaxis.grid(color='gray', linestyle='dashed')
axs[4].plot(omni.index, omni["AE"], color="black", label="AE")
axs[4].plot(omni.index, slow_moving, color="blue", linewidth=2, label="Slow Average")
axs[4].set_ylabel("AE (nT)")
axs[4].set_ylim(bottom=0)
axs[4].legend()

#axs[4].plot(omni.index, f, color="black", label="$|{\delta}AE|$")
#axs[4].set_ylabel("$|{\delta}AE|$ (nT)")
#axs[4].yaxis.grid(color='gray', linestyle='dashed')
#axs[4].xaxis.grid(color='gray', linestyle='dashed')
#axs[4].hlines(y=threshold, xmin=start, xmax=end, color="green", label="Threshold")
#axs[4].hlines(y=-1 * threshold, xmin=start, xmax=end, color="green", label="Threshold")

#axs[4].set_ylim(bottom=0)
#axs[4].legend()

#cbar = plt.colorbar(image_cmap, ax=axs[4])

#cbar.set_label("Bw (pT)\n", loc="center", rotation=270)

fig.set_size_inches(12, 8)

plt.show()

C:\Users\brofe\AppData\Local\Temp\ipykernel_13652\2023986664.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  energy_colors = plt.cm.get_cmap("plasma", num_energies)


A_135702000000000_138855599900000_50000000_10000_TS04D.npz
A_135702000000000_138855599900000_50000000_10000_TS04D.npz
B_135702000000000_138855599900000_50000000_10000_TS04D.npz
B_135702000000000_138855599900000_50000000_10000_TS04D.npz
A_135702000000000_138855599900000_100000000_10000_TS04D.npz
A_135702000000000_138855599900000_100000000_10000_TS04D.npz
B_135702000000000_138855599900000_100000000_10000_TS04D.npz
B_135702000000000_138855599900000_100000000_10000_TS04D.npz
A_135702000000000_138855599900000_300000000_10000_TS04D.npz
A_135702000000000_138855599900000_300000000_10000_TS04D.npz
B_135702000000000_138855599900000_300000000_10000_TS04D.npz
B_135702000000000_138855599900000_300000000_10000_TS04D.npz
A_135702000000000_138855599900000_500000000_10000_TS04D.npz
A_135702000000000_138855599900000_500000000_10000_TS04D.npz
B_135702000000000_138855599900000_500000000_10000_TS04D.npz
B_135702000000000_138855599900000_500000000_10000_TS04D.npz
A_135702000000000_138855599900000_800000000_

In [28]:

f = omni["Bz"].diff()
threshold = 2.0
shifted_f1 = (f-threshold)
shifted_f2 = (f+threshold)

zero_crossings1 = ((shifted_f1.shift() >= 0) & (shifted_f1 < 0)) | ((shifted_f1.shift() < 0) & (shifted_f1 >= 0))
zero_crossings2 = ((shifted_f2.shift() >= 0) & (shifted_f2 < 0)) | ((shifted_f2.shift() < 0) & (shifted_f2 >= 0))
zero_crossings = zero_crossings1 | zero_crossings2
num_crossings = zero_crossings.rolling("1h", center=True).sum().rolling("3h", center=True).mean()

fig, axs = plt.subplots(5, 1, sharex="all")

num_energies = fesa.shape[1]
energy_colors = plt.cm.get_cmap("plasma", num_energies)

for E in range(num_energies):
    
    axs[0].semilogy(fesa.index, fesa[E], label=f"{energies[E]} MeV", color=energy_colors(E), linewidth=2.5)


axs[0].set_ylabel("Flux ($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)")
axs[0].legend(loc="upper right")
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlim(start, end)
axs[0].set_title(f"L = {l_cut}, RBSP-COMBINED")

num_mu = len(mu_vals)
mu_colors = plt.cm.get_cmap("viridis", num_mu)

for k, chosen_mu in enumerate(mu_vals):
    
    selected_psd_refs_A = data_cache.load_psd_cache(satellite = "A", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    selected_psd_refs_B = data_cache.load_psd_cache(satellite = "B", start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    
    epoch_A = selected_psd_refs_A["EPOCH"]
    Lstar_A = selected_psd_refs_A["L_STAR"] 
    L_A = selected_psd_refs_A["L"]
    PSD_A = selected_psd_refs_A["PSD"]
    
    epoch_B = selected_psd_refs_B["EPOCH"]
    Lstar_B = selected_psd_refs_B["L_STAR"]
    L_B = selected_psd_refs_B["L"]
    PSD_B = selected_psd_refs_B["PSD"]
    
    #satisfies_L_cut_A = ((l_cut - tol) < Lstar_A) & (Lstar_A < (l_cut + tol))
    #satisfies_L_cut_B = ((l_cut - tol) < Lstar_B) & (Lstar_B < (l_cut + tol))

    satisfies_L_cut_A = ((l_cut - tol) < L_A) & (L_A < (l_cut + tol))
    satisfies_L_cut_B = ((l_cut - tol) < L_B) & (L_B < (l_cut + tol))
    
    epoch_A_cut = epoch_A[satisfies_L_cut_A]
    PSD_A_cut = PSD_A[satisfies_L_cut_A]
    
    epoch_B_cut = epoch_B[satisfies_L_cut_B]
    PSD_B_cut = PSD_B[satisfies_L_cut_B]
        
    PSD_A_DF = pd.Series(PSD_A_cut, index=epoch_A_cut)
    PSD_A_DF = PSD_A_DF[np.isfinite(PSD_A_DF)]

    PSD_B_DF = pd.Series(PSD_B_cut, index=epoch_B_cut)
    PSD_B_DF = PSD_B_DF[np.isfinite(PSD_B_DF)]

    PSD = pd.concat([PSD_A_DF, PSD_B_DF], sort=False).sort_index()
    PSD = PSD.resample("5h").mean()
    PSD = PSD[np.isfinite(PSD)]
    
    axs[1].semilogy(PSD.index, PSD, linewidth=2, color = mu_colors(k), label=f"{int(chosen_mu)} (MeV / G)")
    
    
axs[1].set_title(rf"RBSP-COMBINED, K = {chosen_k} ($R_E$$G^{{1/2}}$), {field_model.name}")
axs[1].set_ylabel("PSD (c/Mev/G)")
axs[1].legend()

axs[2].yaxis.grid(color='gray', linestyle='dashed')
axs[2].xaxis.grid(color='gray', linestyle='dashed')
axs[2].plot(num_crossings.index, num_crossings, color="black", label="Crossings")
axs[2].set_ylabel("# Crossing Threshold")
axs[2].set_ylim(bottom=0)

axs[3].plot(POES.index, POES["Chorus"], color="black")
axs[3].set_ylabel("Naive Chorus (pT)")
axs[3].set_xlabel("Time")

'''axs[4].plot(omni.index, f, color="black")

axs[4].yaxis.grid(color='gray', linestyle='dashed')
axs[4].xaxis.grid(color='gray', linestyle='dashed')
axs[4].set_ylabel("${\delta}B_{z}$ (nT)")
axs[4].hlines(y=0, xmin=start, xmax=end, color="black")
axs[4].hlines(y=threshold, xmin=start, xmax=end, color="green", label="Threshold")
axs[4].hlines(y=-1 * threshold, xmin=start, xmax=end, color="green", label="Threshold")'''

axs[4].plot(omni.index, omni["Bz"], color="black")
axs[4].plot(omni.index, omni["Bz"].rolling("1h", center=True).mean(), label="Slow Average", color="red", linewidth=2)

axs[4].yaxis.grid(color='gray', linestyle='dashed')
axs[4].xaxis.grid(color='gray', linestyle='dashed')
axs[4].set_ylabel("$B_{z}$ (nT)")
axs[4].set_ylim(-10, 10)
axs[4].legend()

plt.show()



<>:85: SyntaxWarning: invalid escape sequence '\d'
C:\Users\brofe\AppData\Local\Temp\ipykernel_13652\3759418106.py:14: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  energy_colors = plt.cm.get_cmap("plasma", num_energies)
